В этом задании вам необходимо:

Реализовать t-критерий Стьюдента для 2 независимых выборок при условии неизвестной дисперсии. Необходимо считать значение статистики и p_value для разных видов гипотез (двусторонняя, односторонние), а также результат (отвергается нулевая гипотеза или нет). Сравнить результаты с реализацией в scipy.stats.ttest_ind. 

Реализовать bootstrap для оценки (через доверительные интервалы) медианных и средних значений распределений. Рассчитать для распределений: нормальное, экспоненциальное, смесь нормальных (параметры этих распределений на ваш выбор)
Рассчитать мощность критерия для t-критерия и критерия Манна-Уитни для разных распределений и разном эффекте (распределение и эффект на выбор, 2 минимум). 

Оценить корректность t-критерия и критерия Манна-Уитни на 2-х разных распределениях.

Пример распределений: нормальные, смеси нормальных, экспоненциальные, биномиальные. 

Задание принимается в jupyter notebook.

In [1]:
import numpy as np
import pandas as pd
from scipy import stats
from tqdm import tqdm  # Загрузка tqdm для отображения прогресса в циклах

# Функция для бутстрэп оценок медианы и среднего
def bootstrap(data, num_samples, statistic_func, alpha):
    """
    Вычисляет доверительные интервалы для заданной статистики с помощью бутстрепа.

    Args:
        data (array): Исходный массив данных.
        num_samples (int): Количество бутстреп-выборок.
        statistic_func (callable): Функция статистики (например, np.mean или np.median).
        alpha (float): Уровень значимости.

    Returns:
        tuple: Нижняя и верхняя границы доверительного интервала.
    """
    
    # Генерируем множество выборок и вычисляем статистику для каждой выборки
    bootstrapped_stats = [statistic_func(np.random.choice(data, size=len(data))) for _ in range(num_samples)]
    lower = np.percentile(bootstrapped_stats, 100 * alpha / 2)
    upper = np.percentile(bootstrapped_stats, 100 * (1 - alpha / 2))
    
    return lower, upper

# Функция для выполнения t-теста
def ttest_2samp(x1, x2, alternative='two-sided'):
    """
    Выполняет двухвыборочный t-тест Стьюдента для независимых выборок.

    Args:
        x1 (array): Первая выборка наблюдений.
        x2 (array): Вторая выборка наблюдений.
        alternative (str): Определяет альтернативную гипотезу ('two-sided', 'less', 'greater').

    Returns:
        float: t-статистика.
        float: p-значение.
        bool: Результат теста (True если нулевая гипотеза отвергается).
    """
    
    n1, n2 = len(x1), len(x2)
    s1, s2 = x1.var(ddof=1), x2.var(ddof=1)
    x_mean1, x_mean2 = x1.mean(), x2.mean()
    
    # Степени свободы
    df = n1 + n2 - 2
    
    # Объединенная оценка дисперсии
    sp2 = ((n1 - 1) * s1**2 + (n2 - 1) * s2**2) / df
    
    # Стандартная ошибка и t-статистика
    SE = np.sqrt(s1/n1 + s2/n2)
    t_statistic = (x_mean1 - x_mean2) / SE
    dof = (s1/n1 + s2/n2)**2 / (s1**2/(n1**2*(n1-1)) + s2**2/(n2**2*(n2-1)))
   
    
    # Выбор альтернативной гипотезы и расчет p-значения
    if alternative == 'two-sided':
        p_value = stats.t.sf(np.abs(t_statistic), dof) * 2
    elif alternative == 'less':
        p_value = stats.t.cdf(t_statistic, dof)
    elif alternative == 'greater':
        p_value = stats.t.sf(t_statistic, dof)
    
    # Отвержение нулевой гипотезы на уровне значимости 0.05
    reject_null = p_value < 0.05
    
    return t_statistic, p_value, reject_null

# Пример использования функции bootstrap для оценки среднего и медианы для нормального распределения
data_normal = np.random.normal(0, 1, 1000)

# Среднее значение
mean_confidence_interval = bootstrap(data_normal, 10000, np.mean, 0.05)
print(f'Bootstrap confidence interval for the mean (normal distribution): {mean_confidence_interval}')

# Медиана
median_confidence_interval = bootstrap(data_normal, 10000, np.median, 0.05)
print(f'Bootstrap confidence interval for the median (normal distribution): {median_confidence_interval}')


Bootstrap confidence interval for the mean (normal distribution): (-0.11674622096898836, 0.009722654833427227)
Bootstrap confidence interval for the median (normal distribution): (-0.1459209665547903, 0.06463513710995188)


In [2]:
# Симуляция для расчета мощности критерия и проверки корректности теста
def simulate_tests(dist1_params, dist2_params, num_simulations, alpha, effect_size, test_func, *test_args):
    """
    Симулирует тесты для оценки мощности критерия и корректности.

    Args:
        dist1_params (dict): Параметры распределения для первой выборки.
        dist2_params (dict): Параметры распределения для второй выборки.
        num_simulations (int): Количество симуляций.
        alpha (float): Уровень значимости.
        effect_size (float): Размер эффекта для второй выборки.
        test_func (callable): Функция теста для использования (ttest_2samp или mannwhitneyu).
        test_args: Дополнительные аргументы для функции теста.

    Returns:
        float: Оценка мощности критерия.
        float: Оценка корректности критерия.
    """
    # Считаем количество отвергнутых нулевых гипотез и ошибок первого рода
    power_count = 0
    type1_error_count = 0
    for _ in range(num_simulations):
        # Генерация выборок без эффекта и с эффектом
        sample1_no_effect = np.random.normal(**dist1_params)
        sample2_no_effect = np.random.normal(**dist2_params)
        sample2_with_effect = sample2_no_effect + np.random.normal(0, effect_size, dist2_params['size'])

        # Проверка корректности теста (нулевая гипотеза верна)
        _, p_value_no_effect, _ = test_func(sample1_no_effect, sample2_no_effect)
        if p_value_no_effect < alpha:
            type1_error_count += 1

        # Проверка мощности теста (нулевая гипотеза неверна)
        _, p_value_with_effect, _ = test_func(sample1_no_effect, sample2_with_effect)
        if p_value_with_effect < alpha:
            power_count += 1
    
    power_estimate = power_count / num_simulations
    type1_error_estimate = type1_error_count / num_simulations

    return power_estimate, type1_error_estimate

# Примеры распределений
dist1_params = {'loc': 0, 'scale': 1, 'size': 100}
dist2_params = {'loc': 0.5, 'scale': 1, 'size': 100}

# Пример расчета мощности и проверки корректности для t-критерия
num_simulations = 10000
alpha = 0.05
effect_size = 0.5
power_estimate, type1_error_estimate = simulate_tests(
    dist1_params, dist2_params, num_simulations, alpha,
    effect_size, ttest_2samp, 'two-sided'
)
print(f"Estimated power: {power_estimate}")
print(f"Type I error rate: {type1_error_estimate}")

Estimated power: 0.9147
Type I error rate: 0.9413


In [3]:
from scipy.stats import mannwhitneyu

# Функция симуляции для оценки мощности критерия Манна-Уитни и корректности
def simulate_mannwhitneyu(dist1_params, dist2_params, num_simulations, alpha, effect_size):
    """
    Симулирует критерий Манна-Уитни для оценки мощности и корректности.

    Args:
        dist1_params (dict): Параметры распределения для первой выборки.
        dist2_params (dict): Параметры распределения для второй выборки.
        num_simulations (int): Количество симуляций.
        alpha (float): Уровень значимости.
        effect_size (float): Размер эффекта для второй выборки.

    Returns:
        float: Оценка мощности критерия.
        float: Оценка корректности критерия.
    """
    power_count = 0
    type1_error_count = 0
    for _ in tqdm(range(num_simulations)):
        # Генерация выборок без эффекта и с эффектом
        sample1_no_effect = np.random.normal(**dist1_params)
        sample2_no_effect = np.random.normal(**dist2_params)
        sample2_with_effect = sample2_no_effect + np.random.normal(0, effect_size, dist2_params['size'])

        # Проверка корректности теста (нулевая гипотеза верна)
        stat_no_effect, p_value_no_effect = mannwhitneyu(sample1_no_effect, sample2_no_effect, alternative='two-sided')
        if p_value_no_effect < alpha:
            type1_error_count += 1

        # Проверка мощности теста (нулевая гипотеза неверна)
        stat_with_effect, p_value_with_effect = mannwhitneyu(sample1_no_effect, sample2_with_effect, alternative='two-sided')
        if p_value_with_effect < alpha:
            power_count += 1
    
    power_estimate = power_count / num_simulations
    type1_error_estimate = type1_error_count / num_simulations
    
    return power_estimate, type1_error_estimate

# Пример расчета мощности и проверки корректности для критерия Манна-Уитни
alpha = 0.05
effect_size = 0.5  # Примерный размер эффекта, который мы хотим обнаружить
power_estimate_mw, type1_error_estimate_mw = simulate_mannwhitneyu(
    dist1_params,
    dist2_params,
    num_simulations,
    alpha,
    effect_size
)

print(f"Estimated power (Mann-Whitney U): {power_estimate_mw}")
print(f"Type I error rate (Mann-Whitney U): {type1_error_estimate_mw}")

100%|███████████████████████████████████████████████████████████████████████████| 10000/10000 [00:14<00:00, 685.85it/s]

Estimated power (Mann-Whitney U): 0.9013
Type I error rate (Mann-Whitney U): 0.9328


In [4]:
from __future__ import print_function

import numpy as np
from scipy.special import stdtr
from scipy.stats import ttest_ind

a = np.random.randn(40)
b = 4*np.random.randn(40)
t, p = ttest_ind(a, b, equal_var=False)
print("ttest_ind:            t = %g  p = %g" % (t, p))

t1, p1, tf = ttest_2samp(a,b)
print("ttest_2samp:          t = %g  p = %g" % (t1, p1))

# Вычисляем описательную статистику a и b
x_mean1 = np.mean(a)
s1 = np.var(a, ddof=1)
n1 = a.size
adof = n1 - 1

x_mean2 = np.mean(b)
s2 = np.var(b, ddof=1)
n2 = b.size
bdof = n2 - 1
# Используем формулу напрямую
tf = (x_mean1 - x_mean2) / np.sqrt(s1/n1 + s2/n2)
dof = (s1/n1 + s2/n2)**2 / (s1**2/(n1**2*adof) + s2**2/(n2**2*bdof))
pf = 2*stdtr(dof, -np.abs(tf))

print("formula:              t = %g  p = %g" % (tf, pf))

ttest_ind:            t = 2.09618  p = 0.0420757
ttest_2samp:          t = 2.09618  p = 0.0420757
formula:              t = 2.09618  p = 0.0420757
